# Notebook para realizar el estudio PLP
El Patient-Level Prediction (PLP) es una metodología utilizada en el ámbito de la atención médica y la investigación clínica, especialmente dentro del ecosistema de la OMOP Common Data Model (CDM) y la iniciativa OHDSI (Observational Health Data Sciences and Informatics). El propósito de PLP es desarrollar y validar modelos predictivos que puedan anticipar eventos clínicos específicos para pacientes individuales basados en su historial médico y otros datos relevantes.

### Importar librerías

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from src import peticiones, plp, cohortes, miscelania

## Crear cohortes y estudio PLP
Se crean 3 cohortes:

### 1.- Cohorte de Target (Cohort of Interest):

Es el grupo de pacientes que serán estudiados o para los cuales se hará la predicción. Este grupo se define por un conjunto de condiciones o características.
Por ejemplo, podrías tener una cohorte de personas con más de 30 años y que hayan sido diagnosticadas con diabetes.

### 2.- Cohorte de Outcome (Resultado):

Es el evento clínico que se está tratando de predecir en los pacientes del grupo Target. Por ejemplo, si un paciente será readmitido en el hospital dentro de los 30 días después de ser dado de alta, o si desarrollará una enfermedad en el futuro.
El "outcome" es binario en muchos casos (ocurre o no ocurre), pero puede haber diferentes tipos de outcomes en función del problema clínico.

### 3.- Variables Predictorias (Predictive Features):

Son las características del paciente (demográficas, tratamientos, diagnósticos, procedimientos, etc.) que se utilizan para construir el modelo de predicción. Estos datos se obtienen a partir del historial médico del paciente y otras fuentes relevantes, como información sociodemográfica, procedimientos previos, exposiciones a medicamentos, etc.
En PLP, estas características se extraen directamente de la base de datos OMOP.

### 4.- Modelo Predictivo:

El modelo predictivo es el algoritmo que intenta aprender una relación entre las variables predictorias y el outcome. Los algoritmos más comunes en PLP incluyen modelos de regresión logística, árboles de decisión, random forest, redes neuronales, entre otros.
El modelo se entrena utilizando un conjunto de datos histórico (historical cohort) y luego se valida en un conjunto de datos de validación (validation cohort) para evaluar su rendimiento.

### 5.- Validación del Modelo:

Una vez que el modelo ha sido entrenado, se realiza una evaluación en un conjunto de datos independiente para medir su rendimiento. Las métricas más utilizadas en PLP incluyen:
AUC-ROC (Area Under the Receiver Operating Characteristic Curve): mide la capacidad del modelo para distinguir entre los diferentes outcomes.
Precisión, Sensibilidad, Especificidad: evalúan cómo de bien el modelo predice los outcomes esperados.
F1-score: mide el equilibrio entre precisión y sensibilidad.

In [2]:
# Parámetros de ejemplo
db_path = 'data/Eunomia/synthea27nj_5.4_bbdd'

#peticiones.crear_bbdd_desde_csv('data/Eunomia/Synthea27Nj_5.4', 'data/Eunomia/Synthea27Nj_5.4_BBDD')

# Parámetros para la cohorte target
target_tables = ['person', 'VISIT_DETAIL']
target_join_columns = ['person.person_id = VISIT_DETAIL.person_id']
target_conditions = ['person.year_of_birth < 1990']  # Ejemplo: personas mayores de 30 años

# Parámetros para la cohorte outcome
outcome_tables = ['person', 'CONDITION_ERA', 'CONCEPT']
outcome_join_columns = ['person.person_id = CONDITION_ERA.person_id', 'CONDITION_ERA.condition_concept_id = CONCEPT.concept_id']
outcome_conditions = ['CONCEPT.concept_name = \'Acute viral pharyngitis\'']  # Ejemplo: pacientes con Acute viral pharyngitis

# Parámetros para las características predictorias
feature_tables = ['person', 'VISIT_DETAIL', 'DRUG_EXPOSURE']
feature_join_columns = [
    'person.person_id = VISIT_DETAIL.person_id',
    'person.person_id = DRUG_EXPOSURE.person_id'
]
feature_conditions = ['VISIT_DETAIL.visit_detail_start_date BETWEEN \'2019-01-01\' AND \'2023-01-01\'']
feature_columns = ['person.person_id', 'person.year_of_birth', 'VISIT_DETAIL.visit_detail_start_date', 'DRUG_EXPOSURE.drug_concept_id']

# Definir tabla de outcome (target) y condiciones
target_table = 'CONDITION_ERA'
target_join_column = 'person.person_id = CONDITION_ERA.person_id'
target_condition = 'CONDITION_ERA.condition_concept_id = 201826'  # Ejemplo: Diabetes Milletius de tipo 2

# Ejecutar el PLP
results = plp.run_plp_with_algorithms(db_path, target_tables, target_join_columns, target_conditions, outcome_tables, outcome_join_columns, outcome_conditions, 
                                  feature_tables, feature_join_columns, feature_conditions, feature_columns, target_table, target_join_column, target_condition)

Creating target cohort...
Executing query:

    CREATE OR REPLACE TABLE target_cohort AS
    SELECT *
    FROM "person" LEFT JOIN "VISIT_DETAIL" ON person.person_id = VISIT_DETAIL.person_id
    WHERE person.year_of_birth < 1990
    
Target Cohort
   person_id  gender_concept_id  year_of_birth  month_of_birth  day_of_birth  \
0          5               8507           1988               9            22   
1          5               8507           1988               9            22   
2          5               8507           1988               9            22   
3          5               8507           1988               9            22   
4          5               8507           1988               9            22   

  birth_datetime  race_concept_id  ethnicity_concept_id location_id  \
0     1988-09-22             8527              38003563        None   
1     1988-09-22             8527              38003563        None   
2     1988-09-22             8527              38003563    

In [ ]:
miscelania.print_study_summary(feature_columns, target_condition, results)